In [728]:
from selenium import webdriver                  # for browser automation
from splinter import Browser
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service as GeckoService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys # button presses
from selenium.webdriver.common.by import By     # finding elements
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import random
import requests
import re
from pathlib import Path

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Overview

The Kaggle Fragrantica dataset has 22-24k perfumes, accounting for 21-22% of the data available on the website. 

Browsing through the website's search page, notice that:
- Over 50% of the perfumes (57k) were added in the last ten years (2016-2025).
- Over 75% of perfumes were added after the year 2000.
- The search page shows perfumes in batches of 30. You can click "Show more" 33 times, which caps search results at 990=30*33 perfumes per search.
- The key strategy is to filter searches systematically to get group most perfumes into groups of $\leq 990$.
    - Easiest comprehensive partition is by year and gender.
    - Year/gender gives all perfumes from 2000-2009, and all male perfumes from 2000-2025.
    - Starting from 2010, need to add another split since we exceed the 990 cap.
        - "Designers" are disjoint but not well populated.
        - "Notes" are well populated but with huge overlap.
        - Good way forward is using "Notes" to filter with the most popular notes, using a $set()$ call to avoid re-scraping.

In [712]:
fragrantica_overview = pd.read_csv('fragrantica_overview.csv',index_col=0)
fragrantica_overview

,year,unisex,female,male,total,high_rated,cum_total,cum_perc
25,2025,2668,735,365,3768,2352,3768,3.485468
24,2024,5725,1686,874,8285,6077,12053,11.149242
23,2023,4493,1534,867,6894,5111,18947,17.526317
22,2022,4278,1427,826,6531,5077,25478,23.567610
21,2021,3652,1527,829,6008,4628,31486,29.125118
20,2020,3414,1487,852,5753,4382,37239,34.446747
19,2019,3131,1638,732,5501,4559,42740,39.535271
18,2018,2782,1571,819,5172,4264,47912,44.319464
17,2017,2515,1503,710,4728,3767,52640,48.692950
16,2016,2281,1372,609,4262,3568,56902,52.635376


# Scraping search page by year+gender

In [ ]:
# put together once all parts are done
def scrape_general_data(year,gender):
    # compares total number of perfumes by gender
    # and creates a gender:rank dictionary
    gender_counts = fragrantica_overview[ fragrantica_overview['year']==year ][['unisex','female','male']].iloc[0].sort_values(ascending=False)
    gender_ranks = {gender:rank for rank, gender in enumerate(gender_counts.index)}
    
    
    # web scraping search page by year and gender
    
    service = GeckoService(GeckoDriverManager().install())

    with Browser("firefox", service=service, headless=False) as browser:
        # visit search page
        browser.visit("https://www.fragrantica.com/search/")
        
        # find and fill year filters
        browser.find_by_css('input[type="number"]')[1].fill(year)
        time.sleep(1)
        browser.find_by_css('input[type="number"]')[0].fill(year)
        time.sleep(1)
        # use gender:rank dictionary to click on correct gender filter
        browser.find_by_css('input[type="checkbox"]')[gender_ranks[gender]].click()
        time.sleep(2)
        # click Show More button
        num_clicks = min(33,((fragrantica_overview[ fragrantica_overview['year']==year ][gender].iloc[0] - 30)//30)+1)
        for i in range(num_clicks):
            try:
                button = WebDriverWait(browser.driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Show more results')]"))
                )
                browser.driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
                time.sleep(2)
                button.click()
                #browser.find_by_css('button[class="button"]').click()
                time.sleep(2)
            except Exception:
                pass
                
        
        html = browser.html
        soup = BeautifulSoup(html, "html.parser")
    
    # extracts basic data from search page html (url, img, name, brand, gender)

    perfume_data = []

    grid = soup.find('item', class_="grid-x grid-margin-x grid-margin-y small-up-3 medium-up-2 large-up-4 perfumes-row text-center")
    perfumes = grid.find_all('div', class_="cell card fr-news-box", style="flex-grow: 0;")

    for perfume in perfumes:
        img = perfume.find_all('div')[0].img['src']
        url = perfume.find_all('div')[1].a['href']
        name = perfume.find_all('div')[1].a.text
        brand = perfume.find_all('div')[1].small.text
        perfume_data.append((url,img,name,brand, gender))
    
    return pd.DataFrame(data=perfume_data, columns=['url','img','perfume','brand','gender'])

In [ ]:
for year in range(2000,2026):
    df = scrape_general_data(year,'male')
    df.to_csv(f'data/frag_{year}m.csv')
    time.sleep(5)

In [ ]:
for year in range(2000,2013):
    df = scrape_general_data(year,'unisex')
    df.to_csv(f'data/frag_{year}u.csv')
    time.sleep(5)

In [ ]:
for year in range(2000,2010):
    df = scrape_general_data(year,'female')
    df.to_csv(f'data/frag_{year}f.csv')
    time.sleep(5)

# Scraping individual perfume pages

In [ ]:
# Key to read these columns
environments = ['winter','spring','summer','fall','day','night']
longevity_scale = ['very shortlasting','shortlasting','moderate','longlasting','very longlasting']
sillage_scale = ['intimate','moderate','large','enormous']
gender_scale = ['very feminine','slightly feminine','unisex','slightly masculine','very masculine']
price_scale = ['very expensive', 'expensive','average','good value','great value']

In [753]:
def scrape_perfume_firefox(website):

    # Visit specific perfume website and obtain html code
    service = GeckoService(GeckoDriverManager().install())

    with Browser("firefox", service=service, headless=False) as browser:
        # visit search page
        browser.visit(website)
        time.sleep(random.uniform(1.5,2.5))
        # scroll to bottom to some load reviews
        for _ in range(2):
            try:
                browser.find_by_css('a[class="button primary hollow"]').scroll_to()
                time.sleep(random.uniform(0.5,1.5))
                browser.find_by_css('a[class="button primary hollow"]').click()
                time.sleep(random.uniform(0.5,1.5))
            except Exception:
                pass
        html = browser.html
        perfume_soup = BeautifulSoup(html, "html.parser")
        notes_are_categorized = len(browser.find_by_text('Perfume Pyramid'))

    # Extract info from html bit by bit

    pattern = r'([\d.]+)%'  # re pattern for searching

    # accords
    accords = perfume_soup.find_all('div',class_='accord-bar')
    accord_data = []

    try:
        accord_data.extend(
            (accord.text, 
            float(re.search(pattern, accord['style']).group(1)))
            for accord in accords
        )
    except Exception:
        accord_data = []
        
    # notes
    top_notes = []
    mid_notes = []
    bot_notes = []
    uncateg_notes = []

    if notes_are_categorized:
        notes_soup = perfume_soup.find_all('div',style="display: flex; justify-content: center; text-align: center; flex-flow: wrap; align-items: flex-end; padding: 0.5rem;")

        num_notes = [ len(notes_soup[i]) for i in range(3) ]
        top_notes = [ notes_soup[0].find_all('div')[3*i].text for i in range(num_notes[0])]
        mid_notes = [ notes_soup[1].find_all('div')[3*i].text for i in range(num_notes[1])]
        bot_notes = [ notes_soup[2].find_all('div')[3*i].text for i in range(num_notes[2])]
    else:
        notes_soup = perfume_soup.find('div',style="display: flex; justify-content: center; text-align: center; flex-flow: wrap; align-items: flex-end; padding: 0.5rem;")
        num_notes = len(notes_soup)
        uncateg_notes.extend(
            notes_soup.find_all('div')[3*i].text 
            for i in range(num_notes) 
        )
        

    # description
    try:
        description = perfume_soup.find('div',itemprop="description").p.get_text()
    except Exception:
        description = ''

    # rating out of 5
    try:
        rating = perfume_soup.find('span',itemprop='ratingValue').text
        num_votes = int(perfume_soup.find('span',itemprop='ratingCount').text)
    except Exception:
        rating = ''
        num_votes = 0
        
    # user ratings (longevity, sillage, gender, price)
    try:
        ratings = np.array([int(x.text) for x in perfume_soup.find_all('span',class_="vote-button-legend")[11:30]])

        longevity = (100*ratings[:5]/sum(ratings[:5])).round(2)
        sillage = (100*ratings[5:9]/sum(ratings[5:9])).round(2)
        fem_masc = (100*ratings[9:14]/sum(ratings[9:14])).round(2)
        price = (100*ratings[14:19]/sum(ratings[14:19])).round(2)
    except Exception:
        longevity = np.zeros(5)
        sillage = np.zeros(4)
        fem_masc =  np.zeros(5)
        price = np.zeros(5)

    # environment (seasons, day/night
    try:
        environ_soup = perfume_soup.find_all('div',style="width: 100%; height: 0.3rem; border-radius: 0.2rem; background: rgba(204, 224, 239, 0.4);")[8:]
        environ_soup = [str(tag.div) for tag in environ_soup]
        environ_soup = np.array([float(re.search(pattern, tag).group(1)) for tag in environ_soup])
    except Exception:
        environ_soup = np.zeros(6)

    # reviews
    try:
        reviews = perfume_soup.find_all('div',itemprop="reviewBody")
        reviews = [tag.text for tag in reviews]
    except Exception:
        reviews = []
        
    return  pd.DataFrame(data=[{'url':website,'accords':accord_data,
                                    'top_notes':top_notes,'mid_notes':mid_notes,
                                    'bot_notes':bot_notes,'uncateg_notes':uncateg_notes,
                                    'description':description,'rating':rating,
                                    'longevity':longevity,'sillage':sillage,
                                    'fem_masc':fem_masc,'price':price,
                                    'environment':environ_soup,'reviews':reviews}])

In [755]:
def scrape_perfume_chrome(website):

    # Visit specific perfume website and obtain html code
    service = ChromeService(ChromeDriverManager().install())

    with Browser("chrome", service=service, headless=False) as browser:
        # visit search page
        browser.visit(website)
        time.sleep(random.uniform(1.5,2.5))
        # scroll to bottom to some load reviews
        for _ in range(2):
            try:
                browser.find_by_css('a[class="button primary hollow"]').scroll_to()
                time.sleep(random.uniform(0.5,1.5))
                browser.find_by_css('a[class="button primary hollow"]').click()
                time.sleep(random.uniform(0.5,1.5))
            except Exception:
                pass
        html = browser.html
        perfume_soup = BeautifulSoup(html, "html.parser")
        notes_are_categorized = len(browser.find_by_text('Perfume Pyramid'))

    # Extract info from html bit by bit

    pattern = r'([\d.]+)%'  # re pattern for searching

    # accords
    accords = perfume_soup.find_all('div',class_='accord-bar')
    accord_data = []

    try:
        accord_data.extend(
            (accord.text, 
            float(re.search(pattern, accord['style']).group(1)))
            for accord in accords
        )
    except Exception:
        accord_data = []
        
    # notes
    top_notes = []
    mid_notes = []
    bot_notes = []
    uncateg_notes = []

    if notes_are_categorized:
        notes_soup = perfume_soup.find_all('div',style="display: flex; justify-content: center; text-align: center; flex-flow: wrap; align-items: flex-end; padding: 0.5rem;")

        num_notes = [ len(notes_soup[i]) for i in range(3) ]
        top_notes = [ notes_soup[0].find_all('div')[3*i].text for i in range(num_notes[0])]
        mid_notes = [ notes_soup[1].find_all('div')[3*i].text for i in range(num_notes[1])]
        bot_notes = [ notes_soup[2].find_all('div')[3*i].text for i in range(num_notes[2])]
    else:
        notes_soup = perfume_soup.find('div',style="display: flex; justify-content: center; text-align: center; flex-flow: wrap; align-items: flex-end; padding: 0.5rem;")
        num_notes = len(notes_soup)
        uncateg_notes.extend(
            notes_soup.find_all('div')[3*i].text 
            for i in range(num_notes) 
        )
        

    # description
    try:
        description = perfume_soup.find('div',itemprop="description").p.get_text()
    except Exception:
        description = ''

    # rating out of 5
    try:
        rating = perfume_soup.find('span',itemprop='ratingValue').text
        num_votes = int(perfume_soup.find('span',itemprop='ratingCount').text)
    except Exception:
        rating = ''
        num_votes = 0
        
    # user ratings (longevity, sillage, gender, price)
    try:
        ratings = np.array([int(x.text) for x in perfume_soup.find_all('span',class_="vote-button-legend")[11:30]])

        longevity = (100*ratings[:5]/sum(ratings[:5])).round(2)
        sillage = (100*ratings[5:9]/sum(ratings[5:9])).round(2)
        fem_masc = (100*ratings[9:14]/sum(ratings[9:14])).round(2)
        price = (100*ratings[14:19]/sum(ratings[14:19])).round(2)
    except Exception:
        longevity = np.zeros(5)
        sillage = np.zeros(4)
        fem_masc =  np.zeros(5)
        price = np.zeros(5)

    # environment (seasons, day/night
    try:
        environ_soup = perfume_soup.find_all('div',style="width: 100%; height: 0.3rem; border-radius: 0.2rem; background: rgba(204, 224, 239, 0.4);")[8:]
        environ_soup = [str(tag.div) for tag in environ_soup]
        environ_soup = np.array([float(re.search(pattern, tag).group(1)) for tag in environ_soup])
    except Exception:
        environ_soup = np.zeros(6)

    # reviews
    try:
        reviews = perfume_soup.find_all('div',itemprop="reviewBody")
        reviews = [tag.text for tag in reviews]
    except Exception:
        reviews = []
        
    return  pd.DataFrame(data=[{'url':website,'accords':accord_data,
                                    'top_notes':top_notes,'mid_notes':mid_notes,
                                    'bot_notes':bot_notes,'uncateg_notes':uncateg_notes,
                                    'description':description,'rating':rating,
                                    'longevity':longevity,'sillage':sillage,
                                    'fem_masc':fem_masc,'price':price,
                                    'environment':environ_soup,'reviews':reviews}])

# Run webscraping here

In [854]:
already_scraped.shape[0]

76

In [804]:
fragrantica_overview[ fragrantica_overview['year']==2002 ]

,year,unisex,female,male,total,high_rated,cum_total,cum_perc
2,2002,68,207,115,390,369,82269,76.100309


In [855]:
# change as needed
input_csv = 'data/frag_2002/frag_2002f.csv'
output_csv = 'data/frag_2002/frag_2002f_full.csv'

In [857]:
# read in basic info and urls
df = pd.read_csv(input_csv,index_col=0)

# create new csv if starting anew
# else continues an existing partial complete one
if Path(output_csv).exists():
    already_scraped = pd.read_csv(output_csv)
    scraped_urls = set(already_scraped['url'])
else:
    already_scraped = pd.DataFrame()
    scraped_urls = set()

left_to_scrape = df['url'][ ~df['url'].isin(scraped_urls) ]
num_left = df.shape[0]-already_scraped.shape[0]

# scraping starts here
for i in range(0, num_left):
    
    # change browser to not exceed request limits
    change_browser = 0
    
    if i%5==0:
        print(f'URLs remaining:{df.shape[0]-already_scraped.shape[0]}')
    
    website = df['url'].iloc[i]
    
    try:
        if i>0 and i%20==0:
            print('changing browser')
            change_browser += 1
            #time.sleep(random.uniform(90,120))
        if change_browser%2==0:
            scraped_df = scrape_perfume_chrome(website)    
        else:
            scraped_df = scrape_perfume_firefox(website)
    except Exception as e:
        print(f"{website} failed with {e}")
        continue
    
    time.sleep(random.uniform(3,5))
    
    scraped_df.to_csv(output_csv,mode='a', header=not Path(output_csv).exists(),index=False)
    print(f'Scraped and added {already_scraped.shape[0]} out of {df.shape[0]}.')
    already_scraped = pd.read_csv(output_csv)
    scraped_urls = set(already_scraped['url'])

URLs remaining:33
Scraped and added 174 out of 207.
Scraped and added 175 out of 207.
Scraped and added 176 out of 207.
Scraped and added 177 out of 207.
Scraped and added 178 out of 207.
URLs remaining:28
Scraped and added 179 out of 207.
Scraped and added 180 out of 207.
Scraped and added 181 out of 207.
Scraped and added 182 out of 207.
Scraped and added 183 out of 207.
URLs remaining:23
Scraped and added 184 out of 207.
Scraped and added 185 out of 207.
Scraped and added 186 out of 207.
Scraped and added 187 out of 207.
Scraped and added 188 out of 207.
URLs remaining:18
Scraped and added 189 out of 207.
Scraped and added 190 out of 207.
Scraped and added 191 out of 207.
Scraped and added 192 out of 207.
Scraped and added 193 out of 207.
URLs remaining:13
changing browser
Scraped and added 194 out of 207.
Scraped and added 195 out of 207.
Scraped and added 196 out of 207.
Scraped and added 197 out of 207.
Scraped and added 198 out of 207.
URLs remaining:8
Scraped and added 199 out 

In [791]:
already_scraped.tail()

,url,accords,top_notes,mid_notes,bot_notes,uncateg_notes,description,rating,longevity,sillage,fem_masc,price,environment,reviews
63,https://www.fragrantica.com/perfume/Lanman-Kem...,"[('floral', 100.0), ('aquatic', 73.7501), ('oz...",[],[],[],['Lotus'],Lotus & Violets Cologne by Lanman & Kemp is a ...,NaN,[nan nan nan nan nan],[nan nan nan nan],[nan nan nan nan nan],[nan nan nan nan nan],[0. 0. 0. 0. 0. 0.],[]
64,https://www.fragrantica.com/perfume/Al-Jazeera...,"[('amber', 100.0), ('citrus', 100.0), ('lavend...",[],[],[],"['Amber', 'Bergamot', 'Violet', 'Lavender']",Estate by Al-Jazeera Perfumes is a Oriental Fo...,NaN,[nan nan nan nan nan],[nan nan nan nan],[nan nan nan nan nan],[nan nan nan nan nan],[0. 0. 0. 0. 0. 0.],[]
65,https://www.fragrantica.com/perfume/Comme-des-...,"[('amber', 100.0), ('balsamic', 66.9944), ('wa...","['elemi', 'Chamomile', 'Aldehydes']","['French labdanum', 'Spices', 'Ambrette (Musk ...","['Incense', 'Myrrh', 'Olibanum', 'Virginia Ced...",[],Comme des Garcons Series 3 Incense: Avignon by...,NaN,[ 5.71 13.24 45.06 29.65 6.35],[21.36 52.4 18.52 7.72],[ 0.78 1.95 76.95 16.28 4.04],[ 3.7 12.22 60.93 18.97 4.18],[ 97.9029 37.0861 21.0817 100. 65.2318 ...,['This is a REAL REAL church scent (imagine ho...
66,https://www.fragrantica.com/perfume/Frederic-M...,"[('citrus', 100.0), ('green', 76.5877), ('fres...","['Bitter Orange', 'Grapefruit', 'Mandarin', 'T...","['Rose', 'Caraway', 'Neroli', 'Honeysuckle', '...","['Grass', 'Hay', 'Cedar', 'Musk', 'Tonka']",[],Bigarade Concentree by Frederic Malle is a Cit...,NaN,[13.71 26.78 47.87 9.44 2.2 ],[40.52 41.35 9.83 8.29],[ 1.82 4.64 68.16 18.91 6.47],[42.91 33.14 19.35 2.68 1.92],[ 14.6589 70.2467 100. 22.2061 94.0493 ...,"[""In my opinion this is the best hot weather f..."
67,https://www.fragrantica.com/perfume/Ormonde-Ja...,"[('floral', 100.0), ('sweet', 61.5233), ('musk...","['Bamboo', 'Neroli', 'Pink Pepper']","['Champaca', 'Freesia', 'Rice']","['Green Tea', 'Musk', 'Myrhh']",[],Champaca by Ormonde Jayne is a Floral fragranc...,NaN,[ 7.68 21.62 48.89 14.75 7.07],[26.66 46.52 15.85 10.98],[61.25 21.35 15.55 0.7 1.16],[17.34 32.95 42.77 5.2 1.73],[ 18.3333 100. 84.5238 35.2381 91.9048 ...,"['Despite it not being listed in the notes, th..."


# Joining data

In [810]:
fragrantica_overview[ fragrantica_overview['year']==2001 ]

,year,unisex,female,male,total,high_rated,cum_total,cum_perc
1,2001,40,254,105,399,357,82668,76.469391


In [ ]:
df_2001m = pd.read_csv('data/frag_2001/frag_2001m.csv',index_col=0)
df_2001m_full = pd.read_csv('data/frag_2001/frag_2001m_full.csv')
df_2001m_merged = pd.merge(df_2001m,df_2001m_full, how='inner')

df_2001u = pd.read_csv('data/frag_2001/frag_2001u.csv',index_col=0)
df_2001u_full = pd.read_csv('data/frag_2001/frag_2001u_full.csv')
df_2001u_merged = pd.merge(df_2001u,df_2001u_full, how='inner')

df_2001f = pd.read_csv('data/frag_2001/frag_2001f.csv',index_col=0)
df_2001f_full = pd.read_csv('data/frag_2001/frag_2001f_full.csv')
df_2001f_merged = pd.merge(df_2001f,df_2001f_full, how='inner')

df_2001 = pd.concat([df_2001m_merged,df_2001u_merged,df_2001f_merged],
                    axis=0,
                    ignore_index=True)

df_2001 = df_2001.drop_duplicates(ignore_index=True)

In [853]:
df_2001.sample(5)

,url,img,perfume,brand,gender,accords,top_notes,mid_notes,bot_notes,uncateg_notes,description,rating,longevity,sillage,fem_masc,price,environment,reviews
46,https://www.fragrantica.com/perfume/Trussardi/...,https://fimgs.net/mdimg/perfume/m.2158.jpg,Python Uomo,Trussardi,male,"[('woody', 100.0), ('aromatic', 79.836), ('swe...","['Fig Wood Bark', 'Tea']","['Cypress Leaf', 'Olive']","['Tonka Bean', 'Musk', 'Teak Wood', 'Bourbon V...",[],Python Uomo by Trussardi is a Aromatic Fougere...,4.23,[ 8.53 10.85 54.26 20.93 5.43],[19.61 48.37 21.57 10.46],[ 0. 0. 18.18 27.27 54.55],[ 7.89 18.42 52.63 15.79 5.26],[ 11.7647 77.7778 87.5817 26.7974 100. ...,[]
157,https://www.fragrantica.com/perfume/Marc-Jacob...,https://fimgs.net/mdimg/perfume/m.1407.jpg,Marc Jacobs,Marc Jacobs,female,"[('white floral', 100.0), ('tuberose', 50.6382...","['Gardenia', 'Bergamot']","['Tuberose', 'Jasmine', 'Honeysuckle', 'White ...","['Ginger', 'Cedar', 'Musk']",[],Marc Jacobs by Marc Jacobs is a Floral Green f...,NaN,[ 3.44 8.31 50.72 27.22 10.32],[10.7 57.71 20.15 11.44],[68.7 19.08 11.45 0. 0.76],[ 2.68 13.39 58.93 19.64 5.36],[ 29.2566 81.0552 63.5492 39.0887 100. ...,[]
260,https://www.fragrantica.com/perfume/Strenesse/...,https://fimgs.net/mdimg/perfume/m.11291.jpg,Strenesse,Strenesse,female,"[('powdery', 100.0), ('vanilla', 98.7147), ('a...",['Almond'],"['Heliotrope', 'Jasmine', 'Lily-of-the-Valley']","['Vanille', 'Iris', 'Sandalwood', 'Amber']",[],Strenesse by Strenesse is a Floral fragrance f...,3.90,[ 6.12 9.18 36.73 34.69 13.27],[16.28 41.86 26.36 15.5 ],[56.25 31.25 12.5 0. 0. ],[ 0. 0. 54.55 27.27 18.18],[ 78.5714 57.1429 29.5918 65.3061 100. ...,"[""i have the one in this bottle but think i ma..."
214,https://www.fragrantica.com/perfume/Guerlain/P...,https://fimgs.net/mdimg/perfume/m.6324.jpg,Purple Fantasy,Guerlain,female,"[('woody', 100.0), ('citrus', 72.9481), ('powd...","['Coconut', 'Green Tea', 'Bergamot', 'Orange']","['Apricot', 'Jasmine']","['Sandalwood', 'Cedar']",[],Purple Fantasy by Guerlain is a Floral Green f...,3.98,[14.52 22.58 29.03 22.58 11.29],[ 9.57 43.62 21.28 25.53],[70.59 23.53 5.88 0. 0. ],[ 7.14 21.43 57.14 7.14 7.14],[ 67.2414 68.9655 55.1724 98.2759 100. ...,[]
272,https://www.fragrantica.com/perfume/Laura-Merc...,https://fimgs.net/mdimg/perfume/m.4303.jpg,L'Heure Magique,Laura Mercier,female,"[('musky', 100.0), ('powdery', 90.8998), ('ros...","['Bergamot', 'Jasmine']","['White Rose', 'Geranium', 'Spicy Notes']","['Musk', 'Amber', 'Sandalwood']",[],L'Heure Magique by Laura Mercier is a Floral W...,4.15,[18.42 5.26 52.63 10.53 13.16],[11.86 35.59 25.42 27.12],[62.5 12.5 25. 0. 0. ],[ 0. 22.22 44.44 33.33 0. ],[ 84.6154 51.2821 48.7179 89.7436 100. ...,['L’Heure Magique by Laura Mercier is like ste...
